https://www.kaggle.com/muhammete/week-23-homework

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/weather-dataset-rattle-package/weatherAUS.csv


In [3]:
data = pd.read_csv('../input/weather-dataset-rattle-package/weatherAUS.csv')
df = data.copy()

In [4]:
df.shape

(145460, 23)

In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
MinTemp,143975.0,12.194034,6.398495,-8.5,7.6,12.0,16.9,33.9
MaxTemp,144199.0,23.221348,7.119049,-4.8,17.9,22.6,28.2,48.1
Rainfall,142199.0,2.360918,8.478060,0.0,0.0,0.0,0.8,371.0
Evaporation,82670.0,5.468232,4.193704,0.0,2.6,4.8,7.4,145.0
Sunshine,75625.0,7.611178,3.785483,0.0,4.8,8.4,10.6,14.5
WindGustSpeed,135197.0,40.035230,13.607062,6.0,31.0,39.0,48.0,135.0
WindSpeed9am,143693.0,14.043426,8.915375,0.0,7.0,13.0,19.0,130.0
WindSpeed3pm,142398.0,18.662657,8.809800,0.0,13.0,19.0,24.0,87.0
Humidity9am,142806.0,68.880831,19.029164,0.0,57.0,70.0,83.0,100.0
Humidity3pm,140953.0,51.539116,20.795902,0.0,37.0,52.0,66.0,100.0


# Missing Values

**1. Missing values analysis**

Variables with missing values

Percentage of missing values for each variable as a dataframe

Drop missing values in RainToday and RainTomorrow variables

In [6]:
df.isnull().sum()

Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
dtype: int64

In [7]:
# Features with missing values
miss = df.isnull().sum().sort_values(ascending = False).head(15)
miss_per = (miss/len(df))*100

# Percentage of missing values
pd.DataFrame({'No. missing values': miss, '% of missing data': miss_per.values})

,No. missing values,% of missing data
Sunshine,69835,48.009762
Evaporation,62790,43.166506
Cloud3pm,59358,40.807095
Cloud9am,55888,38.421559
Pressure9am,15065,10.356799
Pressure3pm,15028,10.331363
WindDir9am,10566,7.263853
WindGustDir,10326,7.098859
WindGustSpeed,10263,7.055548
Humidity3pm,4507,3.098446


In [8]:
df.dropna(subset=['RainTomorrow','RainToday'], inplace = True)

In [9]:
df.isnull().sum()

Date                 0
Location             0
MinTemp            468
MaxTemp            307
Rainfall             0
Evaporation      59694
Sunshine         66805
WindGustDir       9163
WindGustSpeed     9105
WindDir9am        9660
WindDir3pm        3670
WindSpeed9am      1055
WindSpeed3pm      2531
Humidity9am       1517
Humidity3pm       3501
Pressure9am      13743
Pressure3pm      13769
Cloud9am         52625
Cloud3pm         56094
Temp9am            656
Temp3pm           2624
RainToday            0
RainTomorrow         0
dtype: int64

# 2. Multivariate imputation with (a) selected algorithm(s) and label encoding

Tip: May consider to drop some categorical variables with too many categories, if it takes too much time to get a result

Tip: Do not forget to encode categorical variables.

Tip: You get error if you have data type such as string or date. The values should be integer or float.

In [16]:
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [17]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d')

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140787 entries, 0 to 145458
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Date           140787 non-null  datetime64[ns]
 1   Location       140787 non-null  object        
 2   MinTemp        140319 non-null  float64       
 3   MaxTemp        140480 non-null  float64       
 4   Rainfall       140787 non-null  float64       
 5   Evaporation    81093 non-null   float64       
 6   Sunshine       73982 non-null   float64       
 7   WindGustDir    131624 non-null  object        
 8   WindGustSpeed  131682 non-null  float64       
 9   WindDir9am     131127 non-null  object        
 10  WindDir3pm     137117 non-null  object        
 11  WindSpeed9am   139732 non-null  float64       
 12  WindSpeed3pm   138256 non-null  float64       
 13  Humidity9am    139270 non-null  float64       
 14  Humidity3pm    137286 non-null  float64       
 15  

In [19]:
categories_df = df.select_dtypes(include=['object']).copy()

unique_val = []
for i in categories_df.columns:
    u = categories_df[i].nunique()
    unique_val.append(u)
    
pd.DataFrame({"No. of unique values": unique_val}, index=categories_df.columns)

,No. of unique values
Location,49
WindGustDir,16
WindDir9am,16
WindDir3pm,16
RainToday,2
RainTomorrow,2


# Multivariate Imputation

In [22]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

df_multi_imputation = df.copy()

X = df_multi_imputation.drop(['Location','WindGustDir','WindDir9am','WindDir3pm','Date'], axis=1)
X = pd.get_dummies(X, drop_first=True, columns = ['RainToday','RainTomorrow'], 
                   prefix = ['RainToday','RainTomorrow'])
index = X.index
columns = X.columns

In [23]:
imp_mean = IterativeImputer(random_state=0)
imp_mean.fit(X)
X_imputed = imp_mean.transform(X)
df_imputed_bayesian = pd.DataFrame(X_imputed, index=index, columns=columns)
df_imputed_bayesian.isnull().sum()

/opt/conda/lib/python3.7/site-packages/sklearn/impute/_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)


MinTemp             0
MaxTemp             0
Rainfall            0
Evaporation         0
Sunshine            0
WindGustSpeed       0
WindSpeed9am        0
WindSpeed3pm        0
Humidity9am         0
Humidity3pm         0
Pressure9am         0
Pressure3pm         0
Cloud9am            0
Cloud3pm            0
Temp9am             0
Temp3pm             0
RainToday_Yes       0
RainTomorrow_Yes    0
dtype: int64

# 3. Outliers

Apply Z-score method to detect outliers

Apply IQR method/boxplot visualization for selected variables to detect outliers

Apply Isolation Forest and Local Outlier Factor methods to detect outliers

Tip: Consider standardization based on the algorithm that you use.

Decide on a threshold observation to replace outliers

Replace the outliers with the threshold observation

**Z-score**

In [24]:
from scipy import stats

numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df_numeric = df.select_dtypes(include=numerics)

z = np.abs(stats.zscore(df_numeric))
threshold = 3
print('list of outliers:', '\n', np.where(z > 3)[0], '\n', np.where(z > 3)[1])
print('number of outliers:', np.where(z > 3)[0].shape)

list of outliers: 
 [   292    427    448 ... 140608 140609 140721] 
 [2 2 2 ... 2 2 2]
number of outliers: (2409,)


**IQR**

In [25]:
# IQR values for each variable
Q1 = df_numeric.quantile(0.25)
Q3 = df_numeric.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

MinTemp           9.2
MaxTemp          10.4
Rainfall          0.8
Evaporation       4.8
Sunshine          5.8
WindGustSpeed    17.0
WindSpeed9am     12.0
WindSpeed3pm     11.0
Humidity9am      26.0
Humidity3pm      29.0
Pressure9am       9.4
Pressure3pm       9.6
Cloud9am          6.0
Cloud3pm          5.0
Temp9am           9.3
Temp3pm           9.8
dtype: float64


In [ ]:
# Example: MaxTemp
Q1_MaxTemp = df.MaxTemp.quantile(0.25)
Q3_MaxTemp = df.MaxTemp.quantile(0.75)
IQR_MaxTemp = Q3_MaxTemp - Q1_MaxTemp
upper_MaxTemp = Q3_MaxTemp + 1.5 * IQR_MaxTemp
lower_MaxTemp = Q1_MaxTemp - 1.5 * IQR_MaxTemp
print(f"""
Q1_MaxTemp = {Q1_MaxTemp}
Q3_MaxTemp = {Q3_MaxTemp}
IQR_MaxTemp = {IQR_MaxTemp}
upper_MaxTemp = {upper_MaxTemp}
lower_MaxTemp = {lower_MaxTemp}
""")

In [45]:
df.MaxTemp[df.MaxTemp<lower_MaxTemp].shape

(259,)

In [46]:
df.MaxTemp[df.MaxTemp>upper_MaxTemp].shape

(128,)

In [48]:
import plotly.express as px

fig = px.box(df, y="MaxTemp",)
fig.show()

In [ ]:
# # Example: MaxTemp
# for a in df_numeric.columns:
#     Q1_a = df.a.quantile(0.25)
#     Q3_a = df.a.quantile(0.75)
#     IQR_a = Q3_a - Q1_a
#     upper_a = Q3_a + 1.5 * IQR_a
#     lower_a = Q1_a - 1.5 * IQR_a
#     print(f"""
#     Q1_a = {Q1_a}
#     Q3_a = {Q3_a}
#     IQR_a = {IQR_a}
#     upper_a = {upper_a}
#     lower_a = {lower_a}
#     """)

**Isolation Forest**

In [51]:
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split

df_isolation_forest = df_imputed_bayesian.copy()

y_iso = df_isolation_forest['MaxTemp']
X_iso = df_isolation_forest.drop(['MaxTemp'], axis=1)

clf = IsolationForest(n_estimators=100, max_samples='auto', contamination=0.10, random_state=42)
clf.fit(X_iso)
y_pred = clf.predict(X_iso)

# the model will predict an inlier with a label of +1 and an outlier with a label of -1

outliers_values = X_iso[clf.predict(X_iso) == -1]
outliers_values.shape

(14079, 17)

**Local Outlier Factor**

In [52]:
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import MinMaxScaler

df_lof = df_imputed_bayesian.copy()

scaler = MinMaxScaler()

columns = df_lof.columns

#note that we transform the data with MinMaxScaler
df_lof_scaled = scaler.fit_transform(df_lof)
df_lof_scaled = pd.DataFrame(df_lof, columns=columns)

y = df_lof_scaled['MaxTemp']
X = df_lof_scaled.drop(['MaxTemp'], axis=1)

# fit the model for outlier detection (default)
clf = LocalOutlierFactor(n_neighbors=20, contamination=0.1)

# use fit_predict to compute the predicted labels of the training samples
# (when LOF is used for outlier detection, the estimator has no predict,
# decision_function and score_samples methods).

y_pred = clf.fit_predict(X)
X_scores = clf.negative_outlier_factor_
np.sort(X_scores)[:100]

array([-5.77633953e+10, -5.61517816e+10, -5.57314659e+10, -5.47928518e+10,
       -5.36043770e+10, -5.35059594e+10, -5.33848330e+10, -4.79652148e+10,
       -4.66347142e+10, -4.50336802e+10, -4.43560063e+10, -4.27631527e+10,
       -4.26698522e+10, -4.02131251e+10, -3.94000487e+10, -3.79933724e+10,
       -3.74097230e+10, -3.61324663e+10, -2.97252460e+10, -2.96022383e+10,
       -2.88563949e+10, -2.85108900e+10, -2.76247677e+10, -2.75486265e+10,
       -2.68156339e+10, -2.64762345e+10, -2.62182594e+10, -2.58560763e+10,
       -2.35535275e+10, -2.35038005e+10, -2.28763372e+10, -2.19406414e+10,
       -2.10209966e+10, -2.08210986e+10, -2.00078207e+10, -1.92565790e+10,
       -1.69957882e+10, -1.69326088e+10, -1.64035920e+10, -1.57560398e+10,
       -1.57426887e+10, -1.50520797e+10, -1.44712347e+10, -1.39782334e+10,
       -1.16332202e+10, -8.75574185e+09, -8.05572481e+09, -6.98900923e+09,
       -3.79641058e+09, -2.77285176e+09, -2.04619018e+09, -5.70459307e+00,
       -4.40520691e+00, -

In [53]:
threshold = np.sort(X_scores)[14078]
threshold

-1.1499535039160322

In [54]:
X[(X_scores > threshold)==False].shape

(14079, 17)

# Deleting observations based on the LOF threshold

In [57]:
# Deleting observations based on the LOF threshold
df_numeric_no2 = df_imputed_bayesian[(X_scores > threshold)==True]
print(df_imputed_bayesian.shape)
print(df_numeric_no2.shape)

(140787, 18)
(126708, 18)


In [65]:
#Before deleting
fig = px.box(df, y=df.MaxTemp)
fig.show()

In [63]:
#After deleting
fig = px.box(df_numeric_no2, y=df_numeric_no2.MaxTemp)
fig.show()

In [58]:
# the threshold observation
df_imputed_bayesian[(X_scores == threshold)]

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday_Yes,RainTomorrow_Yes
63499,1.4,11.6,0.8,0.328427,5.566607,50.0,17.0,39.0,86.0,48.0,1022.6,1017.9,2.0,7.0,6.2,10.7,0.0,1.0


In [59]:
threshold_row = df_imputed_bayesian[(X_scores == threshold)]
outliers = df_imputed_bayesian[(X_scores < threshold)]
# We get rid of the indexes of outliers and transformed them into array
outliers.to_records(index = False)
# We define a variable for outlier array
outliers_array = outliers.to_records(index = False)

# We replace all the outliers with the threshold row
outliers_array[:] = threshold_row.to_records(index = False)

df_numeric_no3 = df_imputed_bayesian.copy()

# the outlier observations after replacement by the threshold observation
df_numeric_no3[(X_scores < threshold)] = pd.DataFrame(outliers_array, index = df_numeric_no3[(X_scores < threshold)].index)
df_numeric_no3[(X_scores < threshold)].head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday_Yes,RainTomorrow_Yes
0,1.4,11.6,0.8,0.328427,5.566607,50.0,17.0,39.0,86.0,48.0,1022.6,1017.9,2.0,7.0,6.2,10.7,0.0,1.0
12,1.4,11.6,0.8,0.328427,5.566607,50.0,17.0,39.0,86.0,48.0,1022.6,1017.9,2.0,7.0,6.2,10.7,0.0,1.0
16,1.4,11.6,0.8,0.328427,5.566607,50.0,17.0,39.0,86.0,48.0,1022.6,1017.9,2.0,7.0,6.2,10.7,0.0,1.0
17,1.4,11.6,0.8,0.328427,5.566607,50.0,17.0,39.0,86.0,48.0,1022.6,1017.9,2.0,7.0,6.2,10.7,0.0,1.0
18,1.4,11.6,0.8,0.328427,5.566607,50.0,17.0,39.0,86.0,48.0,1022.6,1017.9,2.0,7.0,6.2,10.7,0.0,1.0


In [64]:
fig = px.box(df_numeric_no3, y=df_numeric_no3.MaxTemp)
fig.show()